In [ ]:
import numpy as np
import pandas as pd

from skimage import io
from skimage.filters import threshold_otsu

import os
import napari
import pickle
import xarray as xr

import morphotrack.points
import morphotrack.flow
import morphotrack.track
import morphotrack.image

In [ ]:
io_directory = '/mnt/ampa_data01/tmurakami/220715_prefrontal_q2_R01/morphotrack'
# load blood vessel
vessel = io.imread(os.path.join(io_directory,'R01_ch561_10um.tif'))

# load nuclei
nuclei = io.imread(os.path.join(io_directory,'R02_ch488_10um.tif'))

# read cellular coordinate
cells_info = pd.read_csv('/mnt/ampa_data01/tmurakami/220715_prefrontal_q2_R01/R02_R01/R02ch488_to_R01_segmentation/coord_centroids.csv',index_col=0) # the unit is original voxel.
scaling_factor = np.asarray([3,0.65,0.65])
cell_coords = cells_info.loc[:,['centroid-0','centroid-1','centroid-2']] * scaling_factor / 10# convert unit to 10 um for consistency
cell_coords = cell_coords.to_numpy()

# load regression model
reg = pickle.load(open(os.path.join(io_directory,'model_debug2.pkl'), 'rb'))
get_norm_flow = morphotrack.flow.flow_to_normflow(reg)

In [ ]:
# selection of the outer most cells. # Note the algorithm does not run in parallel
ver, nor = morphotrack.points.cloud_to_alphashape(cell_coords, downsample=10, alpha=0.1, return_normal=True)

In [ ]:
# selection of the points using clustering
selection = morphotrack.points.face_selection(ver, nor, get_norm_flow(ver), n_clusters=6, cluster_selection='min')
points = ver[selection]

viewer = napari.Viewer()
viewer.add_image(nuclei, rgb=False, name='nuclei', colormap='gray', blending='additive')
viewer.add_points(
    ver,
    size=5,
    face_color = 'green',
    edge_width=0,
    symbol='disc',
)

viewer.add_points(
    points,
    size=5,
    face_color = 'white',
    edge_width=0,
    symbol='disc',
)

## Seeding and tracking

In [ ]:
polynomial_vectorfield = morphotrack.track.polynomial_vectorfield_generator(reg)
tracker = morphotrack.track.FieldTracker(polynomial_vectorfield, points) # make a FieldTracker object

# tracking
t_start = 0
t_end = 500
tracker.solve_ode(t_start,t_end)

In [ ]:
# Visualize tracks
position = tracker.t_positions.stack(pos=['time', 'track']).data.T
position[np.isnan(position)] = 0
flow_vector = abs(get_norm_flow(position))

flow_img = np.zeros(vessel.shape+(vessel.ndim,))
for i in range(vessel.ndim):
    flow_img[...,i] = morphotrack.image.fill_value_in_range(flow_img[...,i],position.T,flow_vector[:,i])
    
viewer = napari.Viewer()
viewer.add_image(vessel, contrast_limits=[0,20000], rgb=False, name='sma', colormap='gray', blending='additive')

palette = ['red','green','blue']

for i in range(vessel.ndim):
    viewer.add_image(flow_img[...,i], contrast_limits=[0,1], rgb=False, name=i, colormap=palette[i], blending='additive')

In [ ]:
tracks = tracker.t_positions
flow = tracker.apply_function_to_position(get_norm_flow)
seeds = tracker.seeds_xr

ds = xr.Dataset(data_vars={'Position': tracks, 'Flow': flow, 'Seed': seeds})

In [ ]:
ds.to_netcdf(os.path.join(io_directory,"track_info.nc"))